In [1]:
import pandas as pd
import os

path = r'D:\elice_python\GAS_5\pytest'

In [2]:
os.chdir(path)

In [3]:
data = pd.read_csv(os.path.join(path, 'ratings.txt'), encoding='cp949', sep='\t', header=0)

In [4]:
data.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        200000 non-null  int64 
 1   document  199992 non-null  object
 2   label     200000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [6]:
data_id=data.iloc[:,0]
data_text=data.iloc[:,1]
data_senti=data.iloc[:,-1]

print(data_text.head())
print(type(data_text))
print('*'*20)
print(data_senti.head())
print(type(data_senti))

0                                  어릴때보고 지금다시봐도 재밌어요ㅋㅋ
1    디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...
2                 폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.
3    와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...
4                          안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.
Name: document, dtype: object
<class 'pandas.core.series.Series'>
********************
0    1
1    1
2    1
3    1
4    1
Name: label, dtype: int64
<class 'pandas.core.series.Series'>


In [7]:
import rhinoMorph

# rhinoMorph 실행
rn = rhinoMorph.startRhino()

morphed_data = ''
for i, data_each in enumerate(data_text):
    morphed_data_each = rhinoMorph.onlyMorph_list(rn, 
                                                  str(data_each), 
                                                  pos=['NNG', 'NNP', 
                                                       'VV', 'VA', 
                                                       'XR', 'IC', 
                                                       'MM', 'MAG', 'MAJ'])
    joined_data_each = ' '.join(morphed_data_each)
    if joined_data_each:
        morphed_data += str(data_id[i]) + ',' + joined_data_each + ',' + str(data_senti[i]) + '\n'


filepath:  C:\Users\Caelu\anaconda3\Lib\site-packages
classpath:  C:\Users\Caelu\anaconda3\Lib\site-packages\rhinoMorph/lib/rhino.jar
RHINO started!


In [8]:
with open('ratings_morphed.txt', 'w', encoding= 'utf8') as f :
    f.write(morphed_data)

In [9]:
data = pd.read_csv('ratings_morphed.txt', encoding='utf8', header=None)
print(data.head())

          0                                                  1  2
0   8112052                              어리 때 보 지금 다시 보 재미있 ㅋㅋ  1
1   8132799  디자인 배우 학생 외국 디자이너 일구 전통 통하 발전 문화 산업 부럽 사실 우리나라...  1
2   4655635                                  폴리스스토리 시리즈 뉴 없 최고  1
3   9251303                    와 연기 진짜 쩔 지루 생각하 몰입 그래 이런 진짜 영화  1
4  10067386                             안개 자욱 하 밤하늘 뜨 초승달 같 영화  1


In [10]:
data_text = data.iloc[:,1]
print(data_text.head())

0                                어리 때 보 지금 다시 보 재미있 ㅋㅋ
1    디자인 배우 학생 외국 디자이너 일구 전통 통하 발전 문화 산업 부럽 사실 우리나라...
2                                    폴리스스토리 시리즈 뉴 없 최고
3                      와 연기 진짜 쩔 지루 생각하 몰입 그래 이런 진짜 영화
4                               안개 자욱 하 밤하늘 뜨 초승달 같 영화
Name: 1, dtype: object


In [11]:
data_senti = data.iloc[:,-1]
print(data_senti.head())

0    1
1    1
2    1
3    1
4    1
Name: 2, dtype: int64


In [12]:
print(type(data_text), type(data_senti))

<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [13]:
from sklearn.model_selection import train_test_split
train_data_text, test_data_text, train_data_senti, test_data_senti = train_test_split(data_text, data_senti, stratify= data_senti, random_state=42)

In [14]:
from collections import Counter
train_data_senti_freq = Counter(train_data_senti)
test_data_senti_freq = Counter(test_data_senti)
print(train_data_senti_freq,test_data_senti_freq)

Counter({1: 74111, 0: 74052}) Counter({1: 24704, 0: 24684})


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(max_features=10000).fit(train_data_text)
X_train = vect.transform(train_data_text)
print('X_train\n', repr(X_train))

X_train
 <148163x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 818547 stored elements in Compressed Sparse Row format>


In [17]:
print(X_train)

  (0, 174)	1
  (0, 6038)	1
  (0, 6754)	1
  (0, 7793)	1
  (1, 4077)	2
  (1, 4529)	2
  (1, 7203)	2
  (1, 7521)	1
  (1, 9477)	1
  (2, 3533)	1
  (2, 3776)	1
  (2, 4371)	1
  (2, 6925)	1
  (2, 7196)	1
  (3, 2369)	1
  (3, 6032)	1
  (3, 7477)	1
  (3, 7874)	1
  (4, 6003)	1
  (5, 921)	1
  (5, 2955)	1
  (5, 3104)	1
  (5, 8111)	1
  (6, 43)	1
  (6, 1670)	2
  :	:
  (148155, 7344)	1
  (148155, 7876)	1
  (148155, 8422)	1
  (148156, 1056)	1
  (148157, 1385)	1
  (148157, 1748)	1
  (148157, 2075)	1
  (148157, 5206)	1
  (148157, 5231)	1
  (148157, 6003)	2
  (148157, 6738)	1
  (148157, 8281)	2
  (148158, 6003)	1
  (148158, 6955)	1
  (148158, 7738)	1
  (148159, 3094)	1
  (148159, 5231)	1
  (148159, 8098)	1
  (148160, 1670)	1
  (148160, 3740)	1
  (148160, 4152)	1
  (148161, 1537)	1
  (148162, 1597)	1
  (148162, 4701)	1
  (148162, 6750)	1
